<a href="https://colab.research.google.com/github/etuckerman/SOCOTEC/blob/main/SOCOTEC_GPU_elliot_tuckerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [2]:
# Check for GPU availability
!nvidia-smi

Mon Dec  9 22:26:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Install required libraries
!pip install -q transformers torch accelerate datasets openai
!pip install -q bitsandbytes

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import re

In [4]:
# Implement function library as specified
def add(a, b): return a + b
def square(a): return a ** 2
def cube(a): return a ** 3
def greet(name): return f"Hello, {name}!"

# Function mapping dictionary
FUNCTION_MAPPING = {
    'add': add,
    'square': square,
    'cube': cube,
    'greet': greet
}

In [5]:
import random
import json

# List of question patterns
add_patterns = [
    "What is {} plus {}?",
    "How much is {} + {}?",
    "What is the sum of {} and {}?",
    "Can you add {} and {} for me?",
    "If I have {} apples and I get {} more, how many do I have in total?"
]

square_patterns = [
    "What is the square of {}?",
    "Can you calculate {} squared?",
    "Find {} squared.",
    "What number do you get when you square {}?",
    "What is {} to the power of 2?"
]

cube_patterns = [
    "What is the cube of {}?",
    "Can you calculate {} cubed?",
    "Find {} cubed.",
    "What number do you get when you cube {}?",
    "What is {} to the power of 3?"
]

greet_patterns = [
    "Say hi to {}.",
    "Greet {}.",
    "Say hello to {}.",
    "Wish {} a good day.",
    "Send a greeting to {}.",
    "Say hi to {} in the morning.",
    "Give a warm greeting to {}.",
    "How about a hello to {}?",
    "Say a quick hello to {}.",
    "Welcome {} to the group!"
]


# Generate 1000 examples for each operation
def generate_add_dataset():
    dataset = []
    for _ in range(1000):
        a = random.randint(1, 100)
        b = random.randint(1, 100)
        pattern = random.choice(add_patterns)
        question = pattern.format(a, b)
        answer = add(a, b)
        dataset.append({'question': question, 'answer': answer})
    return dataset


def generate_square_dataset():
    dataset = []
    for _ in range(1000):
        a = random.randint(1, 100)
        pattern = random.choice(square_patterns)
        question = pattern.format(a)
        answer = square(a)
        dataset.append({'question': question, 'answer': answer})
    return dataset


def generate_cube_dataset():
    dataset = []
    for _ in range(1000):
        a = random.randint(1, 100)
        pattern = random.choice(cube_patterns)
        question = pattern.format(a)
        answer = cube(a)
        dataset.append({'question': question, 'answer': answer})
    return dataset


def generate_greet_dataset():
    dataset = []
    for _ in range(1000):
        name = random.choice(["Alice", "Bob", "Charlie", "Dana", "Eve", "Frank", "Grace", "Hank", "Ivy", "Jack"])
        pattern = random.choice(greet_patterns)
        question = pattern.format(name)
        answer = greet(name)
        dataset.append({'question': question, 'answer': answer})
    return dataset


# Main logic to generate datasets
add_dataset = generate_add_dataset()
square_dataset = generate_square_dataset()
cube_dataset = generate_cube_dataset()
greet_dataset = generate_greet_dataset()

# Combine all the datasets into a single JSON
combined_dataset = {
    "add": add_dataset,
    "square": square_dataset,
    "cube": cube_dataset,
    "greet": greet_dataset
}

# Save to JSON file
with open("synthetic_dataset.json", "w") as f:
    json.dump(combined_dataset, f, indent=4)

print("Dataset successfully created with 1000 examples each for adding, squaring, cubing, and greeting.")


Dataset successfully created with 1000 examples each for adding, squaring, cubing, and greeting.


In [6]:
import json


# Load the synthetic dataset
with open("synthetic_dataset.json", "r") as f:
    with open("synthetic_dataset.json", "r") as f:
        data = json.load(f)

# Flatten the dataset into a single list
flattened_dataset = []
flattened_dataset.extend(data["add"])
flattened_dataset.extend(data["square"])
flattened_dataset.extend(data["cube"])
flattened_dataset.extend(data["greet"])

# Save the flattened dataset to a new JSON file
with open("flattened_dataset.json", "w") as f:
    json.dump(flattened_dataset, f, indent=4)

print("Flattened dataset successfully saved to 'flattened_dataset.json'")


Flattened dataset successfully saved to 'flattened_dataset.json'


In [7]:
# # Comprehensive test cases covering natural and diverse prompts
# test_cases = [
#     # Basic addition examples
#     {"input": "Can you add 7 and 3 for me?", "output": "add(7, 3)"},
#     {"input": "What's the sum of 20 plus 22?", "output": "add(20, 22)"},
#     {"input": "How much is 18 added to 27?", "output": "add(18, 27)"},
#     {"input": "Add 10 and 15 together", "output": "add(10, 15)"},
#     {"input": "Calculate 50 plus 30", "output": "add(50, 30)"},

#     # Square function examples
#     {"input": "Could you find the square of 5?", "output": "square(5)"},
#     {"input": "What is the square of 6?", "output": "square(6)"},
#     {"input": "Please square 7 for me", "output": "square(7)"},
#     {"input": "I need to calculate 8 squared", "output": "square(8)"},
#     {"input": "Can you calculate 4 squared for me?", "output": "square(4)"},

#     # Cube function examples
#     {"input": "What is 2 cubed?", "output": "cube(2)"},
#     {"input": "Can you cube 4 for me?", "output": "cube(4)"},
#     {"input": "Cube the number 3", "output": "cube(3)"},
#     {"input": "What’s the cube of 5?", "output": "cube(5)"},
#     {"input": "Could you find 6 cubed?", "output": "cube(6)"},

#     # Greeting examples
#     {"input": "Greet John for me", "output": "greet('John')"},
#     {"input": "Say hello to Sarah", "output": "greet('Sarah')"},
#     {"input": "Can you greet Emily?", "output": "greet('Emily')"},
#     {"input": "Say hi to Michael", "output": "greet('Michael')"},
#     {"input": "Give John a friendly hello", "output": "greet('John')"},

#     # Multi-function examples
#     {"input": "Add the square of 3 to 10", "output": "add(square(3), 10)"},
#     {"input": "Greet the person who got the cube of 2", "output": "greet('cube(2)')"},
# ]


In [8]:
def format_prompt(input_text):
    return f"""You are an expert at converting natural language to precise Python function calls using ONLY these predefined functions:
- add(a, b): Returns the sum of a and b
- square(a): Returns a squared
- cube(a): Returns a cubed
- greet(name): Returns a greeting for the given name

IMPORTANT: ONLY use these exact function names and signatures.

Convert the following input to EXACTLY the correct function call:
Input: {input_text}
Correct Function Call: """

import ast  # Import ast module for argument parsing

def generate_function_call(model, tokenizer, input_text, max_length=150):
    prompt = format_prompt(input_text)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.1
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Improved regular expression for function call extraction
    function_call_match = re.search(r'Correct Function Call:\s*(\w+\(.*?\))', generated_text)

    if function_call_match:
        function_call = function_call_match.group(1).strip()

        # Preserve original quotes
        # ... (consider keeping quotes as generated)

        # More robust argument parsing using ast.literal_eval
        try:
            args = ast.literal_eval(function_call.split("(")[1].rstrip(")"))
        except (SyntaxError, ValueError):
            args = []

        if not re.match(r'^(add|square|cube|greet)\(', function_call):
            return None

        return function_call
    return None


def execute_function_call(function_call):
    try:
        match = re.match(r'(\w+)\((.*?)\)', function_call)
        if not match:
            return "Invalid function call format"

        func_name, args_str = match.groups()

        # More robust argument parsing
        args = []
        for arg in re.findall(r"'[^']*'|\"[^\"]*\"|[^,\s]+", args_str):
            arg = arg.strip("'\"")
            try:
                arg = int(arg) if arg.isdigit() else arg
            except ValueError:
                pass
            args.append(arg)

        if func_name in FUNCTION_MAPPING:
            return FUNCTION_MAPPING[func_name](*args)
        else:
            return f"Unsupported function: {func_name}"

    except Exception as e:
        return f"Error executing function: {str(e)}"

def evaluate_model(model, tokenizer, test_cases):
    results = []
    for case in test_cases:
        function_call = generate_function_call(model, tokenizer, case['input'])
        expected_call = case['output']

        result = {
            'input': case['input'],
            'generated_call': function_call,
            'expected_call': expected_call,
            'call_match': function_call == expected_call
        }

        if result['call_match']:
            try:
                result['execution_result'] = execute_function_call(function_call)
            except Exception as e:
                result['execution_result'] = str(e)

        results.append(result)

    return results

In [10]:
# Model setup
model_name = "mistralai/Ministral-8B-Instruct-2410"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [12]:
# Set up the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the pad_token_id to the eos_token_id to prevent the warning spam.
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/181k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
# # Run evaluation
# evaluation_results = evaluate_model(model, tokenizer, test_cases)


In [15]:
import json
import random


# Load the flattened dataset
with open("flattened_dataset.json", "r") as f:
    flattened_dataset = json.load(f)

# Randomly select test cases
random.seed(42)  # Ensure reproducibility if needed
sampled_test_cases = random.sample(flattened_dataset, 5)  # Select random test cases

# Map 'question' to 'input' for compatibility with evaluate_model
formatted_test_cases = [
    {"input": case["question"], "output": str(case["answer"])} for case in sampled_test_cases
]

# Run evaluation
evaluation_results = evaluate_model(model, tokenizer, formatted_test_cases)

# Print results
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    print(f"Call Match: {result['call_match']}")
    if result.get('execution_result'):
        print(f"Execution Result: {result['execution_result']}")
    print("---")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input: Can you calculate 57 cubed?
Generated Call: add(2, 3)
Expected Call: 185193
Call Match: False
---
Input: How much is 71 + 66?
Generated Call: square(5)
Expected Call: 137
Call Match: False
---
Input: If I have 34 apples and I get 33 more, how many do I have in total?
Generated Call: square(4)
Expected Call: 67
Call Match: False
---
Input: How about a hello to Hank?
Generated Call: greet("Hank")
Expected Call: Hello, Hank!
Call Match: False
---
Input: Find 35 squared.
Generated Call: add(10, 20)
Expected Call: 1225
Call Match: False
---


In [ ]:
# Print results
for result in evaluation_results:
    print(f"Input: {result['input']}")
    print(f"Generated Call: {result['generated_call']}")
    print(f"Expected Call: {result['expected_call']}")
    print(f"Call Match: {result['call_match']}")
    if result.get('execution_result'):
        print(f"Execution Result: {result['execution_result']}")
    print("---")

In [ ]:
import pandas as pd  # Import DataFrame support


def evaluate_model_with_dataframe(model, tokenizer, test_cases):
    """Evaluates the model's predictions and stores results in a DataFrame."""
    results = []

    # Process each test case
    for case in test_cases:
        function_call = generate_function_call(model, tokenizer, case['input'])
        expected_call = case['output']

        # Check match and execution
        call_match = function_call == expected_call
        execution_result = None

        if call_match:
            try:
                execution_result = execute_function_call(function_call)
            except Exception as e:
                execution_result = str(e)

        # Store results in a dictionary
        results.append({
            "input": case['input'],
            "generated_call": function_call,
            "expected_call": expected_call,
            "call_match": call_match,
            "execution_result": execution_result
        })

    # Convert results to DataFrame
    df = pd.DataFrame(results)

    return df


# Run the evaluation and create the DataFrame
evaluation_df = evaluate_model_with_dataframe(model, tokenizer, test_cases)

# Display results from DataFrame
for index, row in evaluation_df.iterrows():
    print(f"Input: {row['input']}")
    print(f"Generated Call: {row['generated_call']}")
    print(f"Expected Call: {row['expected_call']}")
    print(f"Call Match: {row['call_match']}")
    if row['execution_result'] is not None:
        print(f"Execution Result: {row['execution_result']}")
    print("---")


In [ ]:
evaluation_df.head(10)

In [ ]:
# @title call_match

from matplotlib import pyplot as plt
import seaborn as sns
evaluation_df.groupby('call_match').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)